In [2]:
# ----------------------------------
# 1. Mount Google Drive and Imports
# ----------------------------------

from google.colab import drive
drive.mount('/content/drive')

import os
import asyncio
import nest_asyncio
import openai

import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    mean_absolute_error,
    median_absolute_error,
    explained_variance_score
)

nest_asyncio.apply()  # Allows asyncio.run() in notebooks

# ----------------------------------
# 2. Configure OpenAI Client
# ----------------------------------

# Read your API key from Google Drive
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()

client = openai.AsyncOpenAI(api_key=api_key)

# ----------------------------------
# 3. Load and Preprocess the Dataset
# ----------------------------------

url = "https://raw.githubusercontent.com/apownukepcc/spring-2025-datathon/main/009-Dataset-For-Predictions-With-Specific-Emissions.csv"
df = pd.read_csv(url)

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Define the specific date to be included in the test set later
specific_test_date = pd.to_datetime("2022-07-15")

# Ensure the specific test date is in the dataset
if specific_test_date not in df['date'].values:
    raise ValueError(f"No data available for the date: {specific_test_date}")

# Extract the row for the specific test date and remove it from the main dataset
specific_test_row = df[df['date'] == specific_test_date]
df = df[df['date'] != specific_test_date]

# Define feature columns (weather parameters)
feature_cols = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']

# Define target columns (specific emissions metrics)
target_cols = [
    'SO2TONS_per_LOADMWBA', 'SO2TONS_per_LOADMWBT',
    'NH3TONS_per_LOADMWBA', 'NH3TONS_per_LOADMWBT',
    'NOXTONS_per_LOADMWBA', 'NOXTONS_per_LOADMWBT',
    'COTONS_per_LOADMWBA', 'COTONS_per_LOADMWBT'
]

# Ensure no missing values in features and targets
df = df.dropna(subset=feature_cols + target_cols)

# ----------------------------------
# 4. Train-Test Split
# ----------------------------------

X = df[feature_cols]
y = df[target_cols]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Append the specific test row to the test set (it will be the last row)
X_test = pd.concat([X_test, specific_test_row[feature_cols]])
y_test = pd.concat([y_test, specific_test_row[target_cols]])

# ----------------------------------
# 5. Multi-Output Gradient Boosting
# ----------------------------------
# Wrap GradientBoostingRegressor with MultiOutputRegressor
base_gbr = GradientBoostingRegressor(
    n_estimators=100,   # Number of boosting stages
    learning_rate=0.1,  # Learning rate shrinks contribution of each tree
    max_depth=3,        # Maximum depth of each tree
    random_state=42
)

multi_gbr = MultiOutputRegressor(base_gbr)
multi_gbr.fit(X_train, y_train)

# Predict on the test set
predictions = multi_gbr.predict(X_test)

# ----------------------------------
# 6. Compute Accuracy Metrics
# ----------------------------------

mse = mean_squared_error(y_test, predictions, multioutput='raw_values')
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions, multioutput='raw_values')
mae = mean_absolute_error(y_test, predictions, multioutput='raw_values')
med_ae = median_absolute_error(y_test, predictions, multioutput='raw_values')
explained_var = explained_variance_score(y_test, predictions, multioutput='raw_values')

results = pd.DataFrame({
    'Parameter': target_cols,
    'MSE': mse,
    'RMSE': rmse,
    'R2 Score': r2,
    'MAE': mae,
    'Median AE': med_ae,
    'Explained Variance': explained_var
})

print("Prediction Results with Additional Accuracy Metrics:")
print(results)

# Compare predictions with actual values for the specific test date
pred_df = pd.DataFrame(predictions, index=y_test.index, columns=target_cols)
specific_pred = pred_df.iloc[-1]
specific_actual = y_test.iloc[-1]

comparison_df = pd.DataFrame({
    'Actual': specific_actual,
    'Predicted': specific_pred
})

print(f"\nComparison for specific test date ({specific_test_date.date()}):")
print(comparison_df)

# ----------------------------------
# 7. Prepare Summary for ChatGPT
# ----------------------------------

summary = f"""
Prediction Results with Additional Accuracy Metrics:
{results.to_string(index=False)}

Comparison for specific test date ({specific_test_date.date()}):
{comparison_df.to_string()}

Predictive Method Description:
A Gradient Boosting Regressor (n_estimators=100, learning_rate=0.1, max_depth=3) was wrapped with
MultiOutputRegressor to handle 8 target metrics (SO2, NH3, NOX, CO per LOADMWBA/BT).
Trained on 80% of the dataset using weather features (tavg, tmin, tmax, prcp, snow, wdir, wspd, pres).
The specific date (2022-07-15) was excluded during training, then appended to the test set for a focused prediction comparison.
Performance was evaluated using multiple metrics: MSE, RMSE, R², MAE, Median AE, and Explained Variance.

Question:
Based on the above results and methodology, please provide a comment on the accuracy of these predictions
and any recommendations for improvement.
"""

# ----------------------------------
# 8. Async Function to Query ChatGPT
# ----------------------------------
async def get_chatgpt_comment(user_message):
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_message}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

# ----------------------------------
# 9. Send Query and Print Response
# ----------------------------------
comment_on_accuracy = asyncio.run(get_chatgpt_comment(summary))
print("\n--- ChatGPT Comment on Accuracy ---\n")
print(comment_on_accuracy)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Prediction Results with Additional Accuracy Metrics:
              Parameter           MSE      RMSE   R2 Score       MAE  \
0  SO2TONS_per_LOADMWBA  3.187436e-10  0.000018  -0.238483  0.000013   
1  SO2TONS_per_LOADMWBT  1.843401e-10  0.000014  -3.095830  0.000001   
2  NH3TONS_per_LOADMWBA  1.714838e-07  0.000414  -0.023903  0.000166   
3  NH3TONS_per_LOADMWBT  1.553572e-07  0.000394  -0.071709  0.000032   
4  NOXTONS_per_LOADMWBA  6.712041e-06  0.002591 -13.524711  0.000404   
5  NOXTONS_per_LOADMWBT  6.936560e-06  0.002634 -14.787924  0.000193   
6   COTONS_per_LOADMWBA  4.704782e-06  0.002169 -10.342817  0.000250   
7   COTONS_per_LOADMWBT  4.789052e-06  0.002188 -10.672635  0.000166   

      Median AE  Explained Variance  
0  1.117855e-05           -0.232084  
1  8.397743e-08           -3.094925  
2  1.340192e-04           -0.021477  
3  6.862782e-06  